# Imports

In [51]:
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.initializers import Zeros
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import pickle
from mpl_toolkits.mplot3d import Axes3D

# Creating Dataset

the labels, `y`, can be set to 0 or 1 arbitrarily. This attempts to create a dataset that shows off the model.

In [52]:
np.random.seed(1)
n_samples = 250
X, _ = np.random.uniform(-5, 5, size=(n_samples, 2)), None 

layer1_feature = (np.sin(X[:, 0] * 0.5) + np.cos(X[:, 1] * 0.5)) > 0
layer2_feature = (X[:, 0] ** 2 + X[:, 1] ** 2) > 15
y = (layer1_feature ^ layer2_feature).astype(int)

# Model

Input (2D) -> 2 Hidden Layers (3D) -> Output

The hidden_layer_X_model is used while training to save the hidden spaces from epoch to epoch

In [ ]:
# Build the model using Input layer
inputs = keras.layers.Input(shape=(2,))
hidden1 = keras.layers.Dense(3, activation="sigmoid", name="hidden_layer_1")(inputs)
hidden2 = keras.layers.Dense(3, activation="sigmoid", name="hidden_layer_2")(hidden1)
outputs = keras.layers.Dense(1, activation="sigmoid", name="output_layer")(hidden2)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-1), loss="binary_crossentropy")

hidden_space_trajectory = []
original_points = X  # Store the original points

hidden_layer_1_model = keras.Model(inputs=model.input, outputs=model.get_layer("hidden_layer_1").output)
hidden_layer_2_model = keras.Model(inputs=model.input, outputs=model.get_layer("hidden_layer_2").output)

# Training

This trains the model 1 epoch at a time, and for every interval defined in `epochs_per_interval` records the hidden spaces into `hidden_space_trajectory` as a dictionary for the two layers.

In [48]:
# Train the model for all epochs and record the hidden space every 10 epochs
epochs_per_interval = 10
total_epochs = 300
hidden_space_trajectory = []

for epoch in range(1, total_epochs + 1):
    # Train for one epoch at a time to monitor progress
    model.fit(X, y, epochs=1, verbose=1)

    # Record the hidden space trajectory every 10 epochs
    if epoch % epochs_per_interval == 0:
        # Inspect weights
        weights_hidden1 = model.get_layer("hidden_layer_1").get_weights()[0]
        weights_hidden2 = model.get_layer("hidden_layer_2").get_weights()[0]
        print(f"Weights of hidden layer 1 after {epoch} epochs:\n{weights_hidden1}")
        print(f"Weights of hidden layer 2 after {epoch} epochs:\n{weights_hidden2}")

        hidden1_positions = hidden_layer_1_model.predict(X)
        hidden2_positions = hidden_layer_2_model.predict(X)
        hidden_space_trajectory.append({
            "hidden_layer_1": hidden1_positions,
            "hidden_layer_2": hidden2_positions
        })
        print(f"Recorded hidden space at epoch {epoch}")

# Convert hidden_space_trajectory to numpy array for storage
hidden_space_trajectory = np.array(hidden_space_trajectory)
print(f"Hidden space trajectory shape: {hidden_space_trajectory.shape}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.6918
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6560
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6150 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5643
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5219  
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4984  
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5611
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4640 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5117  
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4384  
Weights of hidden layer 1 after 10 epochs:
[[ 1.4229763  -1.1989154   1.676069  ]
 [ 0.14107329 -0.3964403   2.2638013 ]]
Weights of hidden layer 2 after 10 epochs:
[[ 3.737167    2.9051435  -2.9916112 ]
 [-2.4089217  -2.585308    0.39257503]
 [-1.6081816  -1.1342587   1.8972025 ]]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Recorded hidden space at epoch 10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4574  
8/8 ━━━━━━━━━━━━━━━━━━

# Pickling

This gets the hidden spaces, and pickles them with the original points, and labels for use in blender.

In [50]:
# Save the trajectory and original points to a pickle file
with open("hidden_space_and_points.pkl", "wb") as f:
    pickle.dump({
        "hidden_space_trajectory": hidden_space_trajectory, 
        "original_points": original_points, 
        "labels": y},
        f)

In [ ]:
#debug-this is just to confirm data is being pickled

for epoch_index, hidden_layers in enumerate(hidden_space_trajectory):
    current_frame =  epoch_index

    print(hidden_layers["hidden_layer_1"])

[[4.00277739e-03 9.84956205e-01 9.94755030e-01]
 [5.90959644e-06 9.99980330e-01 1.36066164e-05]
 [3.54093609e-05 9.99950409e-01 1.36962171e-06]
 [8.87913047e-05 9.99782741e-01 8.19161651e-04]
 [2.32656789e-03 9.94199753e-01 6.89283073e-01]
 [3.92926810e-03 9.86543596e-01 9.88870144e-01]
 [2.43787727e-04 9.97772396e-01 9.94803250e-01]
 [1.46435432e-05 9.99882698e-01 8.21128488e-02]
 [3.20205558e-03 9.91990030e-01 8.30739379e-01]
 [3.75453201e-05 9.99930143e-01 1.34911797e-05]
 [5.72803438e-01 1.97530478e-01 1.00000000e+00]
 [8.83768022e-04 9.96068120e-01 9.46594000e-01]
 [7.80022025e-01 1.17458843e-01 1.00000000e+00]
 [1.36492718e-05 9.99980807e-01 1.45724954e-07]
 [1.48973370e-04 9.98527884e-01 9.90758896e-01]
 [2.82739002e-05 9.99897838e-01 1.03774527e-03]
 [8.71650159e-01 1.73511967e-01 9.99957800e-01]
 [1.01861507e-01 9.23520863e-01 6.65503502e-01]
 [1.79331020e-01 6.21911585e-01 9.99995708e-01]
 [1.43955822e-05 9.99855757e-01 3.18045974e-01]
 [9.34603572e-01 5.81615269e-02 9.999997